In [2]:
import geopandas as gpd
import pandas as pd
import pycountry
import countrywrangler as cw


/home/nacho/Documents/MIARFID/PID/.venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
def load_turism_data()-> pd.DataFrame:
    #===================================================================
    # Preproceso de datos turísticos


    turistas = pd.read_csv("data/52047.csv",sep=";")
    turistas["year"] = turistas["Periodo"].apply(lambda x: x[:4])
    turistas["Total"] = turistas["Total"].str.replace(".","")
    turistas["Total"] = turistas["Total"].str.replace(",",".")
    turistas["Total"] = turistas["Total"].str.replace(r'^\s*$', "0", regex=True)
    turistas["Total"] = turistas.Total.fillna(0).astype(float)

    turistas["month"] = turistas["Periodo"].apply(lambda x: x[5:])
    turistas["day"] = 1
    turistas["fecha"] = pd.to_datetime(turistas[["month","year","day"]])
    name_fix = {"Alicante/Alacant":"Alacant",
                "Balears, Illes":"Illes Balears",
                "Castellón/Castelló":"Castelló",
                "Coruña, A":"A Coruña",
                "Rioja, La":"La Rioja",
                "Valencia/València":"València",
                "Gipuzkoa":"Gipuzcoa",
                "Araba/Álava":"Araba",
                "Palmas, Las":"Las Palmas"}
    turistas["Provincia de destino"] = turistas["Provincia de destino"].apply(lambda x: name_fix[x] if x in name_fix else x)
    turistas.drop(columns="RESIDENCIA/ORIGEN",inplace=True)

    prov_geo = 'data/provincias.geojson'
    provs = gpd.read_file(prov_geo)
    turistas = turistas.merge(right=provs[["codigo","provincia"]],right_on="provincia",left_on="Provincia de destino",how="left")
    turistas.codigo.fillna("00",inplace=True)
    # viajeros = turistas[turistas["Concepto turístico"] == "Turistas"].groupby(["year","Provincia de destino"]).sum(numeric_only=True).reset_index()
    # viajeros.rename(columns={"Provincia de destino":"NAMEUNIT"},inplace=True)
    # provincias_viajeros = provincias.merge(right=viajeros,on="NAMEUNIT")
    return turistas

In [7]:
df_turismo = load_turism_data()

/tmp/ipykernel_53643/1684705773.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  turistas = pd.read_csv("https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/52047.csv",sep=";")


KeyError: "['RESIDENCIA/ORIGEN'] not found in axis"

In [5]:
df_turismo

,Continentes,Países,Provincia de destino,Concepto turístico,Periodo,Total,year,month,day,fecha,codigo,provincia
0,NaN,NaN,Total Nacional,Turistas,2023M03,5639935.0,2023,03,1,2023-03-01,00,NaN
1,NaN,NaN,Total Nacional,Turistas,2023M02,4502738.0,2023,02,1,2023-02-01,00,NaN
2,NaN,NaN,Total Nacional,Turistas,2023M01,4602539.0,2023,01,1,2023-01-01,00,NaN
3,NaN,NaN,Total Nacional,Turistas,2022M12,4451187.0,2022,12,1,2022-12-01,00,NaN
4,NaN,NaN,Total Nacional,Turistas,2022M11,5011551.0,2022,11,1,2022-11-01,00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1509700,Oceanía,Otros países o territorios de Oceanía,Zaragoza,Duración media de los viajes,2019M11,0.0,2019,11,1,2019-11-01,50,Zaragoza
1509701,Oceanía,Otros países o territorios de Oceanía,Zaragoza,Duración media de los viajes,2019M10,0.0,2019,10,1,2019-10-01,50,Zaragoza
1509702,Oceanía,Otros países o territorios de Oceanía,Zaragoza,Duración media de los viajes,2019M09,0.0,2019,09,1,2019-09-01,50,Zaragoza
1509703,Oceanía,Otros países o territorios de Oceanía,Zaragoza,Duración media de los viajes,2019M08,0.0,2019,08,1,2019-08-01,50,Zaragoza


In [47]:
df_turismo[df_turismo["Países"].isna() & ~df_turismo["Continentes"].isna()]

,Continentes,Países,Provincia de destino,Concepto turístico,Periodo,Total,year,month,day,fecha,codigo,provincia
7155,Europa,NaN,Total Nacional,Turistas,2023M03,4918485.0,2023,03,1,2023-03-01,00,NaN
7156,Europa,NaN,Total Nacional,Turistas,2023M02,4014623.0,2023,02,1,2023-02-01,00,NaN
7157,Europa,NaN,Total Nacional,Turistas,2023M01,3998560.0,2023,01,1,2023-01-01,00,NaN
7158,Europa,NaN,Total Nacional,Turistas,2022M12,3866828.0,2022,12,1,2022-12-01,00,NaN
7159,Europa,NaN,Total Nacional,Turistas,2022M11,4352808.0,2022,11,1,2022-11-01,00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1395220,Oceanía,NaN,Zaragoza,Duración media de los viajes,2019M11,7.6,2019,11,1,2019-11-01,50,Zaragoza
1395221,Oceanía,NaN,Zaragoza,Duración media de los viajes,2019M10,10.2,2019,10,1,2019-10-01,50,Zaragoza
1395222,Oceanía,NaN,Zaragoza,Duración media de los viajes,2019M09,6.4,2019,09,1,2019-09-01,50,Zaragoza
1395223,Oceanía,NaN,Zaragoza,Duración media de los viajes,2019M08,9.0,2019,08,1,2019-08-01,50,Zaragoza


In [117]:
turismo_por_pais = df_turismo.dropna(subset=["Continentes","Países"])
name_mappings = [(name,cw.Normalize.name_to_alpha2(name)) for name in turismo_por_pais["Países"].unique()]

name_mappings_dict = dict()

for name,code in name_mappings:
    name_mappings_dict[name] = code
name_mappings_dict["República Eslovaca"] = "SK"
name_mappings_dict["Macedonia"] = "MK"
name_mappings_dict["Rumanía"] = "RO"
name_mappings_dict["Estados Unidos de América"] = "US"
name_mappings_dict["Arabia Saudí"] = "SA"
name_mappings_dict["Bahréin"] = "BH"
name_mappings_dict["Corea"] = "KR"
name_mappings_dict["Palestina. Estado Observador, no miembro de Naciones Unidas"] = "PS"

turismo_por_pais["country_code_2"] = turismo_por_pais["Países"].map(name_mappings_dict)


/tmp/ipykernel_6056/270134382.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  turismo_por_pais["country_code_2"] = turismo_por_pais["Países"].map(name_mappings_dict)


In [119]:
turismo_por_pais

,Continentes,Países,Provincia de destino,Concepto turístico,Periodo,Total,year,month,day,fecha,codigo,provincia,country_code_2
14310,Europa,Albania,Total Nacional,Turistas,2023M03,1987.0,2023,03,1,2023-03-01,00,NaN,AL
14311,Europa,Albania,Total Nacional,Turistas,2023M02,1550.0,2023,02,1,2023-02-01,00,NaN,AL
14312,Europa,Albania,Total Nacional,Turistas,2023M01,1853.0,2023,01,1,2023-01-01,00,NaN,AL
14313,Europa,Albania,Total Nacional,Turistas,2022M12,1879.0,2022,12,1,2022-12-01,00,NaN,AL
14314,Europa,Albania,Total Nacional,Turistas,2022M11,2047.0,2022,11,1,2022-11-01,00,NaN,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509700,Oceanía,Otros países o territorios de Oceanía,Zaragoza,Duración media de los viajes,2019M11,0.0,2019,11,1,2019-11-01,50,Zaragoza,None
1509701,Oceanía,Otros países o territorios de Oceanía,Zaragoza,Duración media de los viajes,2019M10,0.0,2019,10,1,2019-10-01,50,Zaragoza,None
1509702,Oceanía,Otros países o territorios de Oceanía,Zaragoza,Duración media de los viajes,2019M09,0.0,2019,09,1,2019-09-01,50,Zaragoza,None
1509703,Oceanía,Otros países o territorios de Oceanía,Zaragoza,Duración media de los viajes,2019M08,0.0,2019,08,1,2019-08-01,50,Zaragoza,None


In [109]:
len(turismo_por_pais["country_code_2"].unique())

195

In [105]:
def load_geojson():
    gdf = gpd.read_file("https://gisco-services.ec.europa.eu/distribution/v2/countries/geojson/CNTR_RG_60M_2020_4326.geojson")
    return gdf

In [107]:
gdf = load_geojson()

In [108]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   id         257 non-null    object  
 1   CNTR_ID    257 non-null    object  
 2   CNTR_NAME  257 non-null    object  
 3   NAME_ENGL  257 non-null    object  
 4   NAME_FREN  242 non-null    object  
 5   ISO3_CODE  257 non-null    object  
 6   SVRG_UN    257 non-null    object  
 7   CAPT       233 non-null    object  
 8   EU_STAT    257 non-null    object  
 9   EFTA_STAT  257 non-null    object  
 10  CC_STAT    257 non-null    object  
 11  NAME_GERM  241 non-null    object  
 12  FID        257 non-null    object  
 13  geometry   257 non-null    geometry
dtypes: geometry(1), object(13)
memory usage: 28.2+ KB


In [111]:
gdf[gdf["NAME_ENGL"] == "Greece"]

,id,CNTR_ID,CNTR_NAME,NAME_ENGL,NAME_FREN,ISO3_CODE,SVRG_UN,CAPT,EU_STAT,EFTA_STAT,CC_STAT,NAME_GERM,FID,geometry
70,EL,EL,Ελλάδα,Greece,Grèce,GRC,UN Member State,Athens,T,F,F,Griechenland,EL,"MULTIPOLYGON (((28.07770 36.11820, 27.86060 35..."
